In [1]:
import numpy as np
import pandas as pd

import os
from scipy.integrate import trapezoid
import pathlib

In [2]:
base_dir = os.path.join(pathlib.Path.home(), 'ecen5407', 'ecen5407_project2')

best_data_pt = os.path.join(base_dir, "data", "test_cases", "Trial_Full_System_90kW_4hr_Battery_with_Geothermal_Ramp_Limits","Trial_Full_System_90kW_4hr_Battery_with_Geothermal_Ramp_Limits_system_info.csv")
best_system_info = pd.read_csv(best_data_pt)

baseline_data_pt = os.path.join(base_dir, "data", "test_cases", "Baseline_System_No_Geothermal","Baseline_System_No_Geothermal_system_info.csv")
baseline_system_info = pd.read_csv(baseline_data_pt)

load_pt = os.path.join(base_dir, "data", "PySam_Outputs", "baseline_system_output.csv")
load = pd.read_csv(load_pt, index_col="Datetime", usecols=["Datetime", "Load (MW)"])
load.set_index(pd.DatetimeIndex(load.index), inplace=True)

In [3]:
x = np.arange(0, 8760, 5/60)
y = np.ravel(load.values)
annual_load = trapezoid(y, x) # MWh
annual_load_kwh = annual_load*1000

In [4]:
annual_load

np.float64(407720.13880330476)

In [108]:
ilr = 1.33
m2_per_wdc = 0.2*0.001
0.29 + 0.04/ilr + 0.16 + 0.18 + m2_per_wdc*(27.07 + 92.11) + 0.044 + 0.02 + 0.03 + m2_per_wdc*2.5 + 0.002

0.7804111879699247

best system

In [21]:
transmission_cost_per_mile = 6000000

geothermal_rating = 87000
geo_binary= 1

geothermal_capex = 775245440
pv_battery_capex = 603005376
wind_capex = 93674384
hybrid_capex = pv_battery_capex + wind_capex

geothermal_op_costs = 118.41*geothermal_rating*geo_binary
wind_op_costs = 30.85*best_system_info["Wind System Size"]
pv_op_costs = 48*best_system_info["PV System Size"] + 180000
hybrid_op_costs = pv_op_costs + wind_op_costs

geo_costs = {'Operating Costs': geothermal_op_costs, 'Capex': geothermal_capex}
hybrid_costs = {'Operating Costs': hybrid_op_costs, 'Capex': hybrid_capex}

system_costs = {}
annual_op_costs =  geothermal_op_costs + hybrid_op_costs
system_costs['Annual Operating Costs'] = annual_op_costs

inflation_rate = 1.0025
annual_op_costs_by_year = [annual_op_costs*inflation_rate**i for i in range(1, 26)]
total_op_costs = sum(annual_op_costs_by_year)
avg_annual_op_costs = total_op_costs/25
system_costs['Average Annual Operating Costs'] = avg_annual_op_costs

upfront_cost = geothermal_capex + hybrid_capex
system_costs['System Capex'] = upfront_cost
avg_annual_upfront_payment = upfront_cost/25
system_costs['Average Annual Capex Payments'] = avg_annual_upfront_payment

avg_annual_total_payment = avg_annual_upfront_payment + avg_annual_op_costs
system_costs['Required Annual Average Revenue'] = avg_annual_total_payment

avg_energy_rate = avg_annual_total_payment/annual_load_kwh
system_costs['Energy Rate (kWh)'] = avg_energy_rate


In [22]:
system_costs

{'Annual Operating Costs': 0    2.340343e+07
 dtype: float64,
 'Average Annual Operating Costs': 0    2.417947e+07
 dtype: float64,
 'System Capex': 1471925200,
 'Average Annual Capex Payments': 58877008.0,
 'Required Annual Average Revenue': 0    8.305648e+07
 dtype: float64,
 'Energy Rate (kWh)': 0    0.20371
 dtype: float64}

In [18]:
best_system_info

,Unnamed: 0,PV System Size,PV AC Capacity,PV AC:DC Ratio,PV System Span,PV Cost,Wind System Size,Wind System Span,Wind Cost,Battery Nominal Power,Battery Capacity,Battery Discharge (Hours),Battery Connection,Battery Charge Efficiency,Battery Discharge Efficiency,Battery Cost,System Cost
0,0,229998.124377,172996.386,1.329497,4031685.21,2.677052e+08,61000.0,9140096.0,89069760.0,89999.9424,359999.7696,4.0,AC,96.0,96.0,1.375199e+08,4.942948e+08


Remove wind case

In [ ]:
transmission_cost_per_mile = 6000000

pv_rating = 320000
geothermal_rating = 87000
geo_binary= 1

wind_capex = 0
wind_op_costs = 0
wind_costs = {'Operating Costs': wind_op_costs, 'Capex': wind_capex}

geothermal_capex = 775245440
geothermal_op_costs = 118.41*geothermal_rating*geo_binary
geo_costs = {'Operating Costs': geothermal_op_costs, 'Capex': geothermal_capex}

pv_capex = 771027840
pv_op_costs = 48*pv_rating + 180000

hybrid_capex = pv_capex
hybrid_op_costs = pv_op_costs
hybrid_costs = {'Operating Costs': hybrid_op_costs, 'Capex': hybrid_capex}

system_costs = {}
annual_op_costs =  wind_op_costs + geothermal_op_costs + hybrid_op_costs
system_costs['Annual Operating Costs'] = annual_op_costs

inflation_rate = 1.0025
annual_op_costs_by_year = [annual_op_costs*inflation_rate**i for i in range(1, 26)]
total_op_costs = sum(annual_op_costs_by_year)
avg_annual_op_costs = total_op_costs/25
system_costs['Average Annual Operating Costs'] = avg_annual_op_costs

real_upfront_cost = geothermal_capex + hybrid_capex
avg_annual_upfront_payment = real_upfront_cost/25
system_costs['System Capex'] = real_upfront_cost
system_costs['Average Annual Capex Payments'] = avg_annual_upfront_payment

avg_annual_total_payment = avg_annual_upfront_payment + avg_annual_op_costs
system_costs['Required Annual Average Revenue'] = avg_annual_total_payment

avg_energy_rate = avg_annual_total_payment/annual_load_kwh
system_costs['Energy Rate (kWh)'] = avg_energy_rate

In [17]:
system_costs

{'Annual Operating Costs': 25841670.0,
 'Average Annual Operating Costs': 26698565.498050522,
 'System Capex': 1546273280,
 'Average Annual Capex Payments': 61850931.2,
 'Required Annual Average Revenue': 88549496.69805053,
 'Energy Rate (kWh)': np.float64(0.21718205276283695)}

baseline

In [12]:
transmission_cost_per_mile = 6000000

pv_rating = 320000
geothermal_rating = 87000
geo_binary = 0

wind_op_costs = 30.85*baseline_system_info["Wind System Size"]
wind_costs = {'Operating Costs': wind_op_costs, 'Capex': wind_capex}

geothermal_cost_per_kwac = 6153.66
geothermal_capex = 26.6*transmission_cost_per_mile*geo_binary + geothermal_cost_per_kwac*87*1000
geothermal_op_costs = 118.41*geothermal_rating*geo_binary
geo_costs = {'Operating Costs': geothermal_op_costs, 'Capex': geothermal_capex}

pv_op_costs = 48*pv_rating + 180000

hybrid_capex = 38814670848
hybrid_op_costs = pv_op_costs
hybrid_costs = {'Operating Costs': hybrid_op_costs, 'Capex': hybrid_capex}

system_costs = {}
annual_op_costs =  wind_op_costs + geothermal_op_costs + hybrid_op_costs
system_costs['Annual Operating Costs'] = annual_op_costs

inflation_rate = 1.0025
annual_op_costs_by_year = [annual_op_costs*inflation_rate**i for i in range(1, 26)]
total_op_costs = sum(annual_op_costs_by_year)
avg_annual_op_costs = total_op_costs/25
system_costs['Average Annual Operating Costs'] = avg_annual_op_costs

real_upfront_cost = geothermal_capex + hybrid_capex
# real_interest_rate = 1 + (0.00614 - 0.0025)
# real_upfront_cost = upfront_cost*real_interest_rate**25 
avg_annual_upfront_payment = real_upfront_cost/25
system_costs['System Capex'] = real_upfront_cost
system_costs['Average Annual Capex Payments'] = avg_annual_upfront_payment

avg_annual_total_payment = avg_annual_upfront_payment + avg_annual_op_costs
system_costs['Required Annual Average Revenue'] = avg_annual_total_payment

avg_energy_rate = avg_annual_total_payment/annual_load_kwh
system_costs['Energy Rate (kWh)'] = avg_energy_rate

In [13]:
system_costs

{'Annual Operating Costs': 0    17421850.0
 Name: Wind System Size, dtype: float64,
 'Average Annual Operating Costs': 0    1.799955e+07
 Name: Wind System Size, dtype: float64,
 'System Capex': 39350039268.0,
 'Average Annual Capex Payments': 1574001570.72,
 'Required Annual Average Revenue': 0    1.592001e+09
 Name: Wind System Size, dtype: float64,
 'Energy Rate (kWh)': 0    3.904642
 Name: Wind System Size, dtype: float64}